### 🟦 Étape 1 : Chargement des bibliothèques et du fichier nettoyé

In [ ]:
# 📦 Import des librairies
import os
import re
import joblib
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix

# 📁 Création des dossiers si nécessaire
os.makedirs("features", exist_ok=True)
os.makedirs("models", exist_ok=True)

# 📂 Chargement des données nettoyées
df = pd.read_csv(
    'data/fiverr_cleaned.csv', # Chemin relatif vers le fichier de données
    encoding='UTF-8', # Utilisé pour décoder les caractères spéciaux (utile si UTF-8 génère des erreurs, fréquent avec des accents)
    low_memory=False  # Force pandas à lire tout le fichier avant de deviner les types de colonnes (évite les avertissements ou erreurs)
)

df["evaluation_parfaite"] = (df["Evaluation"] == 5.0).astype(int)
df["log_rating_count"] = np.log1p(df["Rating_Count"])

print("✅ Données chargées :")
print(df.shape)
df.head()

✅ Données chargées :
(1259, 8)


,Description,Lien,Niveau,Prix,Evaluation,Rating_Count,evaluation_parfaite,log_rating_count
0,"I will do excel formulas, vba macros, charts, ...",https://www.fiverr.com/maltehar/do-excel-formu...,Level 2 Seller,8.701,5.0,56.0,1,4.043051
1,"I will do data mining, web scraping, data extr...",https://www.fiverr.com/hotopilams/do-data-mini...,Top Rated Seller,3.263,5.0,1000.0,1,6.908755
2,"I will do web scraping , data scraping, scrapi...",https://www.fiverr.com/kodweb/do-web-scraping-...,Level 2 Seller,2.176,5.0,63.0,1,4.158883
3,"I will do python web scraping , data entry , a...",https://www.fiverr.com/asad__haadi/do-web-scra...,Level 2 Seller,1.088,5.0,266.0,1,5.587249
4,I will data scraping web site scraping and dat...,https://www.fiverr.com/yourbuddy/web-scraping-...,Level 2 Seller,4.351,4.9,275.0,0,5.620401


In [2]:
# 🔧 Fonction de nettoyage simple
def nettoyer_texte(texte):
    texte = texte.lower()  # passage en minuscules
    texte = re.sub(r'[^\w\s]', '', texte)  # suppression ponctuation
    texte = re.sub(r'\s+', ' ', texte).strip()  # espaces multiples
    return texte

# 🧼 Application du nettoyage sur les colonnes texte
#df["Lien"] = df["Lien"].fillna("").apply(nettoyer_texte)
df["Description"] = df["Description"].fillna("").apply(nettoyer_texte)

### 🟦 Étape 2 : Vectorisation des colonnes textuelles (titre et description)

In [3]:
# 🎯 Vectorisation TF-IDF du titre
#vectoriseur_lien = TfidfVectorizer(max_features=1000)
#X_lien = vectoriseur_lien.fit_transform(df["Lien"])

# 🎯 Vectorisation TF-IDF de la description
vectoriseur_description = TfidfVectorizer(max_features=3000)
X_description = vectoriseur_description.fit_transform(df["Description"].fillna(""))

# 💾 Sauvegarde des vectoriseurs
#joblib.dump(vectoriseur_lien, "models/tfidf_lien.pkl")
joblib.dump(vectoriseur_description, "models/tfidf_description.pkl")

['models/tfidf_description.pkl']

### 🟦 Étape 3 : Encodage des variables catégorielles

In [4]:
# 🎯 Exemple : encodage du niveau du vendeur
encodeur_niveau = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
X_niveau = encodeur_niveau.fit_transform(df[["Niveau"]])

# 💾 Sauvegarde de l’encodeur
joblib.dump(encodeur_niveau, "models/ohe_niveau.pkl")

['models/ohe_niveau.pkl']

In [5]:
# 📊 Extraction des colonnes transformées
X_evaluation_bin = csr_matrix(df[["evaluation_parfaite"]].values)
X_log_rating = csr_matrix(df[["log_rating_count"]].values)

# 💾 Sauvegarde des nouvelles variables
joblib.dump(df[["evaluation_parfaite"]].values, "models/evaluation_parfaite.pkl")
joblib.dump(df[["log_rating_count"]].values, "models/log_rating_count.pkl")

['models/log_rating_count.pkl']

### 🟦 Étape 4 : Assemblage de toutes les features

In [6]:
# 🧱 Fusion des features en une seule matrice
X_final = sparse.hstack([X_description, X_niveau, X_evaluation_bin, X_log_rating])

# ✅ Vérification du nombre de lignes
assert X_final.shape[0] == df.shape[0], "Erreur : le nombre de lignes de X_final ne correspond pas à celui du dataframe d'origine."

# 💾 Sauvegarde au format sparse
sparse.save_npz("features/features_fiverr.npz", X_final)

print("Shape finale des features :", X_final.shape)


Shape finale des features : (1259, 1327)


### 🟦 Étape 5 (optionnelle) : Sauvegarde de la target si applicable

In [7]:
# 🎯 Exemple : sauvegarde du prix comme target si disponible
if "Prix" in df.columns:
    y = df["Prix"].values
    joblib.dump(y, "models/y_prix.pkl")
    print("Target sauvegardée")

Target sauvegardée


In [8]:
# 📦 Chargement de la target
y = df["Prix"].values  # ou: joblib.load("models/y_prix.pkl")

'''
# ✂️ Séparation entraînement/test
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

# 💾 Sauvegarde
joblib.dump(X_train, "train_test/X_train.pkl")
joblib.dump(X_test, "train_test/X_test.pkl")
joblib.dump(y_train, "train_test/y_train.pkl")
joblib.dump(y_test, "train_test/y_test.pkl")
'''

'\n# ✂️ Séparation entraînement/test\nX_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)\n\n# 💾 Sauvegarde\njoblib.dump(X_train, "train_test/X_train.pkl")\njoblib.dump(X_test, "train_test/X_test.pkl")\njoblib.dump(y_train, "train_test/y_train.pkl")\njoblib.dump(y_test, "train_test/y_test.pkl")\n'